In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config
set_config(transform_output='pandas')
from sklearn.impute import SimpleImputer

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate,StratifiedKFold, cross_val_score, train_test_split
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder,MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor

In [15]:
#read data scikit-learn
os.chdir('C:/Users/Katharina/Desktop/Weiterbildung/Bootcamp/Bootcamp/Final_project/data')
#data = pd.read_csv('DatenML_V2_relCol.csv', sep=';')
data = pd.read_csv('DatenML_V2_relCol_ZusatzType.csv', sep=';')	

In [16]:
df = data
# Erstellen der neuen Spalte "Team1":
# Wenn TeamDesignation = A, dann soll TeamAName in Team1 stehen,
# andernfalls (also bei B) TeamBName.
df["Team1"] = np.where(
    df["TeamDesignation"].str.upper() == "A",
    df["@TeamAName"],
    df["@TeamBName"]
)

# Erstellen der Spalte "Team2" als das jeweils andere Team:
df["Team2"] = np.where(
    df["TeamDesignation"].str.upper() == "A",
    df["@TeamBName"],
    df["@TeamAName"]
)

In [32]:
#Gender umdefinieren
df['Gender_x'] = df['Gender_x'].astype('object')

In [33]:
#bestimmte Variablen entfernen
# data1 = df.drop(['FirstName', 'LastName', 'TeamDesignation', '@PointsTeamASet3', '@PointsTeamBSet3', 
#        'FederationCode_y', '@TeamAName', '@TeamBName','Gender_x','TeamName',
#        '@LocalDate', '@LocalTime', 'temperature_2m',
#        'precipitation', 'wind_speed_10m', 'rain', 'wind_gusts_10m',
#         'SpikeTotal',
#         '@DurationSet3'], axis=1)
data1 = df.drop(['FirstName', 'LastName', 'TeamDesignation','@PointsTeamASet1', '@PointsTeamBSet1', '@PointsTeamASet2',
       '@PointsTeamBSet2', '@PointsTeamASet3', '@PointsTeamBSet3', 
       'FederationCode_y', '@TeamAName', '@TeamBName',
       '@DurationSet1','@LocalDate', '@LocalTime',
       #neu
       'FirstName2', 'LastName2','Type','TournamentNo','NoPlayer1_team', 'NoPlayer2_team',#'total_A', 'total_B', 'TeamName'-> nicht mehr da
       '@DurationSet2', '@DurationSet3',
        'temperature_2m',
        #angepasst für den Fall
        'precipitation', 'wind_speed_10m', 'rain', 'wind_gusts_10m',
         'SpikeTotal',], axis=1)
#'Gender_x',

In [34]:
y = data1.pop('SpikeFault')
X = data1.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31416)

In [26]:
df.to_csv('ML_SpikeFault_OHNEWetter_V2.csv', index=False, sep=';')

In [35]:
# select categorical and numerical column names
nominal_features = X.select_dtypes(include=['object']).columns.tolist()
# Define feature types
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Create preprocessors for different feature types
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler())
])

nominal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Build the column transformer
transformers = [
    ('num', numeric_transformer, numeric_features),
    ('nom', nominal_transformer, nominal_features)
]

preprocessor = ColumnTransformer(transformers=transformers)

In [21]:
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

model_scores = []
def score_model(model_name, y_true, y_pred):
  scores = {
      'Model': model_name,
      'MAE ($)': round(mean_absolute_error(y_true, y_pred), 2),
      'RMSE ($)': round(root_mean_squared_error(y_true, y_pred), 2),
      'MAPE (%)': round(100 * mean_absolute_percentage_error(y_true, y_pred), 2),
      'R-Squared': round(r2_score(y_true, y_pred), 3)
  }
  return scores

In [37]:
from sklearn.ensemble import GradientBoostingRegressor
gbr_pipeline = make_pipeline(preprocessor,
                             RobustScaler(),
                             GradientBoostingRegressor(random_state=42, n_estimators=200, learning_rate=0.1, 
                                                  max_depth=5, subsample=0.8))

GB = gbr_pipeline.fit(X_train, y_train)

gbr_predictions = gbr_pipeline.predict(X_test)
model_scores.append(score_model('GradientBoostigRegressor', y_test, gbr_predictions))
pd.DataFrame(model_scores)

,Model,MAE ($),RMSE ($),MAPE (%),R-Squared
0,GradientBoostigRegressor,2.21,2.88,4.078567e+16,0.432
1,RandomForest,2.24,2.92,4.096973e+16,0.418
2,GradientBoostigRegressor,2.21,2.87,4.006514e+16,0.437


In [38]:
import joblib
# Modell speichern als .pkl-Datei
joblib.dump(GB, "GradientBoosting_SpikeFault_OHNEWetter_V2.pkl")

['GradientBoosting_SpikeFault_OHNEWetter_V2.pkl']

In [39]:
from sklearn.ensemble import RandomForestRegressor
rf_pipeline = make_pipeline(preprocessor,
                             RobustScaler(),
                             RandomForestRegressor(random_state=42, n_estimators=200, max_depth=15, 
                                         min_samples_split=5, min_samples_leaf=2))

RF = rf_pipeline.fit(X_train, y_train)

rf_predictions = rf_pipeline.predict(X_test)

model_scores.append(score_model('RandomForest', y_test, rf_predictions))
pd.DataFrame(model_scores)

,Model,MAE ($),RMSE ($),MAPE (%),R-Squared
0,GradientBoostigRegressor,2.21,2.88,4.078567e+16,0.432
1,RandomForest,2.24,2.92,4.096973e+16,0.418
2,GradientBoostigRegressor,2.21,2.87,4.006514e+16,0.437
3,RandomForest,2.24,2.92,4.120575e+16,0.417


In [40]:
import joblib
# Modell speichern als .pkl-Datei
joblib.dump(RF, "RandomForest_SpikeFault_OHNEWetter_V2.pkl")

['RandomForest_SpikeFault_OHNEWetter_V2.pkl']